In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import os
from tkinter import messagebox
from datetime import datetime
import calendar

warnings.filterwarnings('ignore')

def obtener_fecha(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    mes_nombre = calendar.month_abbr[mes].upper()[:3] # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
    fecha = f"{año}{str(mes).zfill(2)}" # 202401
    
    return mes_año, fecha

mes_año, fecha = obtener_fecha(mes_anterior=True)

asignacion_path = f'bases/asignacion/{fecha}/base_asignacion_20241002.xlsx'
recupero_path = f'bases/recupero/{fecha}/Recupero_{fecha}.xlsx'
cancelaciones_path = f'bases/cancelaciones/{fecha}/Cancelaciones {mes_año}.xlsx'

final_path = os.path.abspath(cancelaciones_path)

print('Base Asignacion:', asignacion_path)
print('Base Recupero:',recupero_path)
print('Base Cancelaciones:', cancelaciones_path)

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar stock recupero?', icon='warning')
if result == 'yes':
    df_recupero = pd.read_excel(recupero_path, sheet_name='BASE')
    print('Base Recupero:', df_recupero.shape)

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar base asignacion?', icon='warning')
if result == 'yes':
    df_asignacion = pd.read_excel(asignacion_path, sheet_name='Hoja1')
    print('Base Asignación:', df_asignacion.shape)

root.destroy()

In [ ]:
df_asignacion_test = df_asignacion.copy()
df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)
df_recupero_test = df_recupero.copy()
df_recupero_test.columns = clean_columns(df_recupero_test.columns)

In [ ]:
df_asignacion_test['CODIGO'] = df_asignacion_test['CODIGO'].astype('Int64').astype(str).str.zfill(8)
df_asignacion_test['CONT_18'] = df_asignacion_test['CONT_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

df_asignacion_test['TIPO_FONDO'].replace({'REACTIVA_KST': 'REACTIVA'}, inplace=True)
df_asignacion_test['TIPO_FONDO'].fillna('NULL', inplace=True)

df_asignacion_test['CAMP'] = df_asignacion_test['CAMP'].str.extract(r'(\d+)%').astype(float)
df_asignacion_test['CAMP'].fillna(0, inplace=True)

df_asignacion_test['IMPORTE_CAMP'] = df_asignacion_test.apply(lambda x: x['CAPITAL'] if x['TIPO_FONDO'] in ['REACTIVA', 'FAE', 'CRECER'] else x['CAPITAL']*x['CAMP']/100, axis=1)

cols_asignacion = ['CODIGO', 'CONT_18', 'CLAVE', 'IMPORTE_CAMP', 'TIPO_FONDO']
df_asignacion_test = df_asignacion_test[cols_asignacion]


print('Base Asignación:', df_asignacion_test.shape)
df_asignacion_test.head(5)

In [ ]:
df_recupero_test.rename(columns={'CODCENTRAL': 'CODIGO', 'CONT_SAE': 'CONT_18', 'ENTIDAD': 'CLAVE'}, inplace=True)

df_recupero_test = df_recupero_test[df_recupero_test['BANCA'] == 'MINORISTA']
df_recupero_test = df_recupero_test[df_recupero_test['TIPO_CARTERA'] != 'SECURED']

df_recupero_test['CLAVE'] = df_recupero_test['CLAVE'].replace('KST', 'KSTBC')

df_recupero_test['CODIGO'] = df_recupero_test['CODIGO'].astype('Int64').astype(str).str.zfill(8)
df_recupero_test['CONT_18'] = df_recupero_test['CONT_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)

df_recupero_test['LIQUIDO'].fillna(0, inplace=True)
df_recupero_test['TRANSACCION_REFINANCIADO'].fillna(0, inplace=True)
df_recupero_test['TOTAL'] = df_recupero_test['LIQUIDO'] + df_recupero_test['TRANSACCION_REFINANCIADO']

df_grouped = df_recupero_test.groupby('CONT_18')['TOTAL'].sum().reset_index()

df_recupero_test = df_recupero_test.drop_duplicates(subset='CONT_18', keep='last')
df_recupero_test = df_recupero_test.merge(df_grouped, on='CONT_18', suffixes=('', '_sum'))

df_recupero_test['TOTAL'] = df_recupero_test['TOTAL_sum']
df_recupero_test.drop(columns='TOTAL_sum', inplace=True)

df_recupero_test.sort_values(['CODIGO', 'CONT_18', 'CLAVE', 'TOTAL'], inplace=True)
df_recupero_test.reset_index(drop=True, inplace=True)

cols_recupero = ['CODIGO', 'CONT_18', 'CLAVE', 'TOTAL']
df_recupero_test = df_recupero_test[cols_recupero]

print('Base Recupero:', df_recupero_test.shape)
df_recupero_test.head(5)

In [ ]:
df_cancelaciones = df_recupero_test.merge(df_asignacion_test[['CONT_18', 'IMPORTE_CAMP', 'TIPO_FONDO']], on='CONT_18', how='left')
df_cancelaciones['CUMPLE'] = np.where(df_cancelaciones['TOTAL'] >= df_cancelaciones['IMPORTE_CAMP'], 'SI', 'NO')
df_cancelaciones['TOTAL'] = df_cancelaciones['TOTAL'].round(2)
df_cancelaciones['IMPORTE_CAMP'] = df_cancelaciones['IMPORTE_CAMP'].round(2)

df_cancelaciones = df_cancelaciones[df_cancelaciones['CUMPLE'] == 'SI']

df_cancelaciones.rename(columns={'TOTAL': 'IMPORTE_SUMA', 'IMPORTE_CAMP': 'IMPORTE_MINIMO'}, inplace=True)
df_cancelaciones = df_cancelaciones[['CODIGO', 'CONT_18', 'CLAVE', 'IMPORTE_SUMA', 'IMPORTE_MINIMO', 'CUMPLE', 'TIPO_FONDO']]

df_cancelaciones.sort_values(['IMPORTE_SUMA'], ascending=False, inplace=True)
df_cancelaciones.reset_index(drop=True, inplace=True)
df_cancelaciones.to_excel(final_path, index=False, sheet_name=f'CANCELACIONES {mes_año}')

df_cancelaciones.head(5)

In [ ]:
os.startfile(final_path)